In [1]:
!pip install geopy folium pandas openpyxl
from geopy.geocoders import Nominatim
import pandas as pd
import folium
import time

In [2]:
# Load the Excel file
data = pd.read_excel(r"C:\Users\demet\OneDrive\Desktop\Scrubbed Helene Data_PRINT.xlsx")

In [3]:
geolocator = Nominatim(user_agent="geo_app")

In [7]:
# Define a function to geocode an address with a delay
def geocode_address(address):
    try:
        location = geolocator.geocode(address)
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except:
        return None, None

In [9]:
# Create Latitude and Longitude columns by applying the geocode function with delay
latitudes = []
longitudes = []

In [13]:
data['Latitude'] = latitudes
data['Longitude'] = longitudes

In [15]:
# Save the updated data back to Excel
output_file = 'geocoded_addresses.xlsx'
data.to_excel(output_file, index=False)
print(f"Geocoding complete. Data saved to {output_file}")

Geocoding complete. Data saved to geocoded_addresses.xlsx


In [17]:
lat_median = data['Latitude'].median()
lon_median = data['Longitude'].median()

def assign_quadrant(lat, lon):
    if lat >= lat_median and lon >= lon_median:
        return 'NE'
    elif lat >= lat_median and lon < lon_median:
        return 'NW'
    elif lat < lat_median and lon >= lon_median:
        return 'SE'
    else:
        return 'SW'

data['Quadrant'] = data.apply(lambda row: assign_quadrant(row['Latitude'], row['Longitude']), axis=1)

In [23]:
# Center map around the median location
map_center = [data['Latitude'].median(), data['Longitude'].median()]
m = folium.Map(location=map_center, zoom_start=13)

# Assign colors to each quadrant
quadrant_color = {'NE': 'blue', 'NW': 'green', 'SE': 'orange', 'SW': 'red'}

# Add markers to the map
for _, row in data.dropna(subset=['Latitude', 'Longitude']).iterrows():
    folium.Marker(
        [row['Latitude'], row['Longitude']],
        popup=row['Post Address'],
        icon=folium.Icon(color=quadrant_color.get(row['Quadrant'], 'gray'))
    ).add_to(m)

# Save the map
m.save('map_with_quadrants.html')
print("Map created successfully and saved as 'map_with_quadrants.html'")

Map created successfully and saved as 'map_with_quadrants.html'
